### A simple sample of application provided here

In [16]:
# import the source code

from EasyBldLoad import * # May take a while to load


In [17]:
bldLoad = csv2TimeSeries(csvPath='SampleData_UCSD/BuildingLoadAgg.csv', 
                         timeColumn='DateTime', valueColumn='RealPower')

In [18]:
bldLoad # You can have an overview of the data 

<TimeSeries (DataArray) (DateTime: 110879, component: 1, sample: 1)> Size: 887kB
array([[[517.251]],

       [[530.934]],

       [[525.827]],

       ...,

       [[695.311]],

       [[661.281]],

       [[640.733]]])
Coordinates:
  * DateTime   (DateTime) datetime64[ns] 887kB 2017-01-01T00:15:00 ... 2020-0...
  * component  (component) object 8B 'RealPower'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

#### darts.Timeseries provides a better structured way of storing time series with related features, 
#### you can refer to https://unit8co.github.io/darts/generated_api/darts.timeseries.html for detailed usage

In [19]:
# only first 2 months data used for training
trainingData = bldLoad.drop_after(pd.Timestamp('2017-03-01'))

In [20]:
trainingData

<TimeSeries (DataArray) (DateTime: 5663, component: 1, sample: 1)> Size: 45kB
array([[[517.251]],

       [[530.934]],

       [[525.827]],

       ...,

       [[711.154]],

       [[712.99 ]],

       [[709.333]]])
Coordinates:
  * DateTime   (DateTime) datetime64[ns] 45kB 2017-01-01T00:15:00 ... 2017-02...
  * component  (component) object 8B 'RealPower'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [21]:
lags_past = []
lags_future = []
for i in range(1, 25):
    lags_past.append(-i*2)
    lags_future.append(i*2)

In [22]:
predictor = EasyBldLoad(forecastMethod='XGBForecast',
                        loadData = trainingData, # model will be trained on the whole data
                            modelParams={
                                'lags_past' : lags_past,
                                'lags_future' : lags_future,
                                'adaptiveLags' : False, # Please set this to False, since it is a special feature for my own case
                                'historicalMaxWindowDays' : 60, # This is the window size for the historical max value, useful for long term online training and forecasting
                                'otherParams' : {} # Put other XGB parameters here
                            },
                            generalSettings={
                                'platformInfo' : 'MyLaptop_i7_13700',
                                'muteRuntimeNotification' : False
                            },
                            saveSettings={
                                'saveModel' : True,
                                'savePath' : 'SavedModels/',
                                'savePrediction' : True
                            }
                        )

In [23]:
predictor.fit()

Training the model...


/opt/anaconda3/envs/Citylearn/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [01:27:58] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


In [24]:
# dataprovided here should be the same format as the training data
# the model itself is in predictor.model, you can also call other prediction methods
# refer to the documentation for more prediction methods : https://unit8co.github.io/darts/generated_api/darts.models.forecasting.xgboost.html

prediction = predictor.predict(trainingData,96) # default prediction method, predict n steps after the end of the provided data

/Users/lunlongli/Documents/GitHub/EasyBldLoad/EasyBldLoad.py:239: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.predictionLog = pd.concat(


In [25]:
prediction # predictions are delivered in the same format as the training data

<TimeSeries (DataArray) (DateTime: 96, component: 1, sample: 1)> Size: 384B
array([[[ 692.5724 ]],

       [[ 686.02655]],

       [[ 669.52905]],

       [[ 667.86633]],

       [[ 660.4426 ]],

       [[ 654.908  ]],

       [[ 656.3338 ]],

       [[ 640.1481 ]],

       [[ 649.57355]],

       [[ 645.1796 ]],

...

       [[ 878.8388 ]],

       [[ 866.6038 ]],

       [[ 872.00635]],

       [[ 839.2238 ]],

       [[ 821.6792 ]],

       [[ 821.3446 ]],

       [[ 812.1921 ]],

       [[ 697.71216]],

       [[ 693.1268 ]],

       [[ 711.47577]]], dtype=float32)
Coordinates:
  * DateTime   (DateTime) datetime64[ns] 768B 2017-03-01 ... 2017-03-01T23:45:00
  * component  (component) object 8B 'RealPower'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [26]:
predictionMAPE = mape(prediction, bldLoad) # Calculate the MAPE of the prediction
predictionMAE = mae(prediction, bldLoad) # Calculate the MAE of the prediction
predictionRMSE = rmse(prediction, bldLoad) # Calculate the RMSE of the prediction
print('MAPE: {:.2f}%'.format(predictionMAPE))
print('MAE: {:.2f}'.format(predictionMAE))
print('RMSE: {:.2f}'.format(predictionRMSE))

MAPE: 1.93%
MAE: 16.37
RMSE: 20.84


In [27]:
# make another prediction
prediction2 = predictor.predict(trainingData,96*2) # default prediction method, predict n steps after the end of the provided data

In [28]:
predictionMAPE2 = mape(prediction2, bldLoad) # Calculate the MAPE of the prediction
predictionMAE2 = mae(prediction2, bldLoad) # Calculate the MAE of the prediction
predictionRMSE2 = rmse(prediction2, bldLoad) # Calculate the RMSE of the prediction
print('MAPE: {:.2f}%'.format(predictionMAPE2))
print('MAE: {:.2f}'.format(predictionMAE2))
print('RMSE: {:.2f}'.format(predictionRMSE2))

MAPE: 2.48%
MAE: 20.85
RMSE: 26.19


#### if savePrediction is set True, after each forecast, prediction is saved as both csv and pickle for future usage
#### if saveModel is set True, after fitting the model is saved as pickle and will be automatically loaded if save model is needed (the model name is now distinguish by only training sets start and end time, if params changed please specify different save path)

In [29]:
predictor.forecaster.getLog()

{'trainingStartTime': Timestamp('2017-01-01 00:15:00'),
 'trainingEndTime': Timestamp('2017-02-28 23:45:00'),
 'fitComsumedTimeSeconds': 13.476361989974976,
 'saveModeld': True,
 'modelID': 'XGBModel_201701010015_201702282345',
 'modelPath': 'SavedModels//model',
 'modelSize': 24914713,
 'platformInfo': 'MyLaptop_i7_13700'}

## Happy forecasting !